In [1]:
import torch
import os
import cv2
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import copy
from Data_PreProcess import Data_preprocess as dp
import kfbReader as kr
from Net import Net
from Generate_Anchor import generate_prior_anchor, get_prior_anchor
prior_anchor = get_prior_anchor()
import json
import time
model = Net()
model.load_state_dict(torch.load(r"E:\cancer\model\temp\eps=2.t7"),strict = False)
model = model.cuda()
model.eval()
print("load")

load


In [ ]:
data_path = r"E:\cancer\train_data"
gt = pd.read_csv(r"E:\cancer\imglabel\val\3_gt.csv",index_col=0)
datalist = list(gt.index)
label = dp.get_gt(gt)
input_size = 1024
model.eval()
TP = []
FP = []
FN = []
with torch.no_grad():
    for idx,dat_name in enumerate(datalist):
        start = time.time()
        src_img = cv2.imread(os.path.join(data_path, dat_name))
        height, width = src_img.shape[:2]
        ans = []
        for h in range(0,height,input_size):  #后续改成 d = self.input_size/2 
            h = h if h+input_size < height else height - input_size
            for w in range(0,width,input_size):
                w = w if w+input_size < width else width - input_size
                #img = reader.ReadRoi(w,h,input_size,input_size,20)
                img = src_img[h:h+input_size, w:w+input_size]
                img = img.transpose(2,0,1)[np.newaxis,]
                img = torch.from_numpy(img).cuda().float()
                pred_cls,pred_loc = model(img)
                index = pred_cls>0.5
                if torch.sum(index) > 0:
                    scores = pred_cls[index]
                    pred_loc = pred_loc[index]
                    src_anchor = prior_anchor[index]
                    x = pred_loc[:,0] * src_anchor[:,2] + src_anchor[:,0] + w
                    y = pred_loc[:,1] * src_anchor[:,3] + src_anchor[:,1] + h
                    _w = torch.exp(pred_loc[:,2]) * src_anchor[:,2]
                    _h = torch.exp(pred_loc[:,3]) * src_anchor[:,3]
                    pred = torch.stack((x,y,_w,_h,scores),axis=1).detach().cpu().numpy()
                    pred = np.clip(pred,0,None)
                    index = dp.nms(pred)
                    bboxs = pred[index].astype(np.float)
                    ans.append(bboxs)
        if len(ans)==0:continue
        ans = np.vstack(ans)
        tp,fp,fn = dp.AP(ans, label[dat_name])
        TP.append(tp)
        FP.append(fp)
        FN.append(fn)
print("final")